In [1]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
tf_dtype = tf.dtypes.float32
import os
import sklearn
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
import pickle


from lioness import Lioness
from panda import Panda
import pandas as pd


In [2]:
import dataloader
import utils
#utils.set_random_seed(1)

from NOTEARS import NOTEARS
from ContextualNOTEARS import ContextualNOTEARS
from LowRankContextualNOTEARS import LRContextualNOTEARS


In [3]:
def gen_data(data_params):
    if data_params["use_archetypes"]:
        W_dict, C_dict = dataloader.gen_archetypes(data_params["d"], data_params["n_edges"],
                                                         data_params["n_c"], data_params["k_true"],
                                                         graph_type=data_params["graph_type"],
                                                         ensure_convex=data_params["ensure_convex"])
        sample_loadings, W, C, X = dataloader.gen_samples(
            W_dict, C_dict, n=data_params["n"], n_i=data_params["n_i"], n_mix=data_params["n_mix"],
            sem_type=data_params["sem_type"])
        for i in range(data_params["n_c"]):
            C[:, i] += np.random.normal(0.,
                                        ((1-data_params["context_snr"])/data_params["context_snr"])*np.var(C[:, i]),
                                        size=C[:, i].shape)
    else:
        W, C, X = dataloader.gen_samples_no_archs(data_params["n"], data_params["d"],
                                                                  data_params["n_edges"], data_params["n_i"],
                                                                  data_params["n_c"], 
                                                                  c_signal_noise=data_params["context_snr"],
                                                                  graph_type=data_params["graph_type"],
                                                                  sem_type=data_params["sem_type"])
        W_dict, C_dict = None, None
    return W, C, X, W_dict, C_dict

def get_f1s(W_test, W_test_hat, threshs):
    return [
        np.mean([utils.f1_mat(W_test[i], W_test_hat[i], thresh, thresh) for i in range(len(W_test))])
            for thresh in threshs
    ]

In [4]:
threshs = [0.0, 0.001, 0.01, 0.02, 0.03, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

def correlation_from_covariance(covariance):
    v = np.sqrt(np.diag(covariance))
    outer_v = np.outer(v, v)
    correlation = covariance / outer_v
    correlation[covariance == 0] = 0
    return correlation



# the below is redundant but kept for reference.  

data_params = {
    "n": 100,    # number of DAGs
    "n_i": 1,     # number of samples per DAG
    "d" : 16,       # number of vertices in each DAG
    "n_edges": 4,     # expected number of edges in each DAG
    "n_c": 8,    # number of contextual features
    "use_archetypes": True,
    "ensure_convex" : False, # should the archetype be generated such that they form a convex set of DAGs?
    "k_true" : 2,       # number of true archetypes
    "graph_type": "ER",
    'sem_type' : 'gauss',
    'context_snr' : 1.0, # signal-to-noise ratio of the contextual data
    "n_mix": 2
}

In [5]:
#####   NON CONVEX CASE    ####### 

os.makedirs("results", exist_ok=True)
with open("results/c_notears_paramsweepNonConvNaive.csv", 'w') as out_file:
    print("n\tn_edges\tn_c\tcontext_snr\tk_true\tk", end="\t", file=out_file)
    print("\t".join(["base_{:.3f}".format(x) for x in threshs]), end="\t", file=out_file)
    print("\t".join(["notears_{:.3f}".format(x) for x in threshs]), end='\t', file=out_file)
    print("\t".join(["lioness_{:.3f}".format(x) for x in threshs]), end='\t', file=out_file)
    print("\t".join(["c_notears_{:.3f}".format(x) for x in threshs]), end='\t', file=out_file)
    print("\t".join(["lr_c_notears_{:.3f}".format(x) for x in threshs]), file=out_file)
    
    #for arch in range(2, 20):  #number of archetypes
    data_params = {
"n": 100,    # number of DAGs
"n_i": 1,     # number of samples per DAG
"d" : 16,       # number of vertices in each DAG
"n_edges": 4,     # expected number of edges in each DAG
"n_c": 8,    # number of contextual features
"use_archetypes": True,
"ensure_convex" : False, # should the archetype be generated such that they form a convex set of DAGs?
"k_true" : 2,       # number of true archetypes
"graph_type": "ER",
'sem_type' : 'gauss',
'context_snr' : 1.0, # signal-to-noise ratio of the contextual data
"n_mix": 2
}

##### This is where the blocked comments began. 
##### THIS IS WHERE I BEGIN MASSIVE INDENT  ##################

    W, C, X, W_dict, C_dict = gen_data(data_params)

    ####### THIS IS ALL OF THE LIONESS STUFF.  #######
    ###### Make sure that the squeeze and split is consistent across models  ########

    ####### I am moving all of this squeezing and unsqueezing and Dataframing South Of the Split  #######



    C_train, C_test, X_train, X_test, W_train, W_test = train_test_split(C, X, W, test_size=0.25)

    ##### # Start Copy From Above   ######
    X = np.squeeze(X)
    X = pd.DataFrame(X)

    #print(X)

    C = pd.DataFrame(C)
    ####### # End Copy From Above
    print("this should all be the same as before")
    #assert False


    C_test =pd.DataFrame(C_test)   # I think this is where the hitch was 
    index = C_test.index
    a_list = list(index)
    print(a_list)


    ##### this is all from test_lioness   ########
    ppi            ='puma/ToyData/ToyPPIData.txt'
    motif          ='puma/ToyData/ToyMotifData.txt'
    expression_data='puma/ToyData/ToyExpressionData.txt'
    lioness_file   ='Travis'
    rm_missing     =False
    output_file    ='panda.npy'
    gt_file        ='panda/test_panda.txt'
    #panda_obj      =Panda(expression_data, motif, ppi, save_tmp=True, remove_missing=rm_missing,
    #                  keep_expression_matrix=bool(lioness_file), modeProcess='legacy', save_memory=False)
    # Set parameters


    #2. Testing Lioness with motif set to None to compute Lioness on coexpression networks
    motif          = None
    
    # Make sure to keep epxression matrix for next step
    panda_obj      = Panda(expression_data, motif, ppi, save_tmp=True, remove_missing=rm_missing,
                      keep_expression_matrix=True, modeProcess='legacy')
    lioness_obj    = Lioness(panda_obj, data_params, X, a_list, start=1, end=1)
   
    lioness_obj.save_lioness_results(lioness_file)

    res  = np.load('lioness_output/lioness.1.npy')
    gt   = np.load('lioness/lionessCoexpression.1.npy')
    W_OGLioness = pickle.load(open('lionessParams.pkl', 'rb'))

    W_OGLioness = np.array(W_OGLioness)
    
    ##### This is where the block comments ended



    #W, C, X, W_dict, C_dict = gen_data(data_params)
    #C_train, C_test, X_train, X_test, W_train, W_test = train_test_split(C, X, W, test_size=0.25)

    notears = NOTEARS({'l1': 1e-3, 'alpha': 1e2, 'rho':1e1, 'gamma':1e1},
                     (data_params["n_c"], 1),
                     (data_params["d"], data_params["d"]))
    print("Fitting Population Model...")
    notears.fit(C_train, X_train, epochs=10, batch_size=1)
    print("Finished Fitting Population Model.")
    pop_model = notears.model.predict(np.expand_dims(C_train[0], 0)).squeeze()


    def not_i(ar, i):
        if i == 0:
            return ar[1:]
        if i == len(ar) - 1:
            return ar[:-1]
        return np.vstack((ar[:i], ar[i+1:]))
    def lioness():
        results = []
        for i in range(data_params["n"]):
            print("{} / {}".format(i, data_params['n']), end='\r')
            notears.model.set_weights([pop_model]) # initialize at pop model
            notears.fit(not_i(C_train, i), not_i(X_train, i), epochs=3, batch_size=1, verbose=0)
            results.append((data_params["n"]-1)*(
                notears.model.predict(np.expand_dims(C_train[0], 0)).squeeze() - pop_model))
        return results

    f1s_lioness   = get_f1s(W_test, W_OGLioness, threshs)
    f1s_baseline = get_f1s(W_test, np.ones_like(W_test), threshs)
    C_test = C_test.to_numpy() 
    f1s_notears  = get_f1s(W_test, [notears.model.predict(np.expand_dims(c, 0)).squeeze() for c in C_test], threshs)

 

    rank = 5
    for k in range(1, 20):
        print("k={}".format(k), end='\r')
        model_params = {
            "k": k,     # Learned archetype dictionary size
            "encoder_input_shape": (data_params["n_c"], 1),
            "encoder_output_shape": (k,),
            "dict_shape": (k, data_params["d"], data_params["d"]),
            "dict_output_shape": (data_params["d"], data_params["d"]),
            "sample_specific_loss_params": {'l1': 1e-3, 'alpha': 1e2, 'rho': 1e1, 'gamma': 1e1},
            "archetype_loss_params": {'l1': 0, 'alpha': 0, 'rho': 0},
            "learning_rate": 1e-3
        }
        print("{}\t{}\t{}\t{:.3f}\t{}\t{}".format(data_params["n"], data_params["n_edges"], data_params["n_c"],
                                     data_params["context_snr"], data_params["k_true"], 
                                      model_params["k"]), end='\t')

        lr_model_params = {
            "k": k,     # Learned archetype dictionary size
            "encoder_input_shape": (data_params["n_c"], 1),
            "encoder_output_shape": (k,),
            "dict_shape": (k, data_params["d"], rank),
            "dict_output_shape": (data_params["d"], data_params["d"]),
            "sample_specific_loss_params": {'l1': 1e-3, 'alpha': 1e2, 'rho': 1e1, 'gamma': 1e1},
            "archetype_loss_params": {'l1': 0, 'alpha': 0, 'rho': 0},
            "learning_rate": 1e-3
        }
        lr_c_notears = LRContextualNOTEARS(lr_model_params["encoder_input_shape"],
                                      lr_model_params["encoder_output_shape"],
                                      lr_model_params["dict_shape"],
                                     lr_model_params["sample_specific_loss_params"],
                                     lr_model_params["archetype_loss_params"],
                                     lr_model_params["learning_rate"])
        lr_c_notears.fit(C_train, X_train, batch_size=1, epochs=50)

        c_notears = ContextualNOTEARS(model_params["encoder_input_shape"],
                                      model_params["encoder_output_shape"],
                                      model_params["dict_shape"],
                                     model_params["sample_specific_loss_params"],
                                     model_params["archetype_loss_params"],
                                     model_params["learning_rate"])
        c_notears.fit(C_train, X_train, batch_size=1, epochs=50)

        # Measure recovery of sample-specific networks
        W_test_pred = [c_notears.model.predict(np.expand_dims(c, 0)).squeeze() for c in C_test]
        f1s = get_f1s(W_test, W_test_pred, threshs)
        W_test_pred_lr = [lr_c_notears.model.predict(np.expand_dims(c, 0)).squeeze() for c in C_test]
        f1s_lr = get_f1s(W_test, W_test_pred_lr, threshs)
        print("{}\t{}\t{}\t{:.3f}\t{}\t{}".format(data_params["n"], data_params["n_edges"], data_params["n_c"],
                                     data_params["context_snr"], data_params["k_true"], 
                                      model_params["k"]), end='\t', file=out_file)
        print("\t".join(["{:.3f}".format(x) for x in f1s_baseline]), end='\t', file=out_file)
        print("\t".join(["{:.3f}".format(x) for x in f1s_notears]), end='\t', file=out_file)
        print("\t".join(["{:.3f}".format(x) for x in f1s_lioness]), end='\t', file=out_file)
        print("\t".join(["{:.3f}".format(x) for x in f1s]), end='\t', file=out_file)
        print("\t".join(["{:.3f}".format(x) for x in f1s_lr]), file=out_file)
        
        



this should all be the same as before
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Loading expression data ...
  Elapsed time: 0.15 sec.
Loading PPI data ...
Number of PPIs: 238
  Elapsed time: 0.16 sec.
Calculating coexpression network ...
  Elapsed time: 0.05 sec.
Returning the correlation matrix of expression data in <Panda_obj>.correlation_matrix
Loading input data ...
  Elapsed time: 0.00 sec.
(1000, 50)
(100, 16)
let's get this straight
[[ 1.00000000e+00  4.01338605e-02  4.97832033e-01  5.04314703e-02
  -2.28601882e-01  3.65281077e-02  3.42723079e-02  3.09459862e-02
  -1.59635102e-01 -1.09800849e-01  1.63866377e-02 -2.54013540e-03
  -1.00838723e-02 -5.52887041e-02  7.00147269e-01  1.72933594e-01]
 [ 4.01338605e-02  1.00000000e+00  5.11784853e-02  2.60316596e-02
   1.15005690e-01 -3.01491394e-02  4.91558896e-02 -1.98787836e-03
  -2.21534591e-01  1.43925217e-03  2.04708662e-02  5.71654378e-03
   6.55544474e-03 -4.63618313e-02  4.6735528

  Elapsed time: 0.12 sec.
BOOOOOOOOM
Running LIONESS for sample 10:
Computing coexpression network:
howdy one
(16, 100)
are these indices okay?
(16, 99)
(16, 16)
  Elapsed time: 0.01 sec.
Normalizing networks:
  Elapsed time: 0.00 sec.
Inferring LIONESS network:
here?
  Elapsed time: 0.00 sec.
did we get here boo boo 2?
Saving LIONESS network 10 to lioness_output using npy format:
  Elapsed time: 0.13 sec.
BOOOOOOOOM
Running LIONESS for sample 11:
Computing coexpression network:
howdy one
(16, 100)
are these indices okay?
(16, 99)
(16, 16)
  Elapsed time: 0.01 sec.
Normalizing networks:
  Elapsed time: 0.00 sec.
Inferring LIONESS network:
here?
  Elapsed time: 0.00 sec.
did we get here boo boo 2?
Saving LIONESS network 11 to lioness_output using npy format:
  Elapsed time: 0.18 sec.
BOOOOOOOOM
Running LIONESS for sample 12:
Computing coexpression network:
howdy one
(16, 100)
are these indices okay?
(16, 99)
(16, 16)
  Elapsed time: 0.01 sec.
Normalizing networks:
  Elapsed time: 0.00 s

/home/stsou/miniconda2/envs/tarrLab/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


DDDDDIIIIIDDDD WE GET THROUGH THE F1 SCORES?
100	4	8	1.000	2	1	Epoch 1/50
75/75 [==============================] - 0s 1ms/step - loss: 209.4254
Epoch 2/50
75/75 [==============================] - 0s 1ms/step - loss: 147.2389
Epoch 3/50
75/75 [==============================] - 0s 1ms/step - loss: 123.9518
Epoch 4/50
75/75 [==============================] - 0s 1ms/step - loss: 112.2851
Epoch 5/50
75/75 [==============================] - 0s 1ms/step - loss: 105.2448
Epoch 6/50
75/75 [==============================] - 0s 991us/step - loss: 100.5700
Epoch 7/50
75/75 [==============================] - 0s 998us/step - loss: 97.1314
Epoch 8/50
75/75 [==============================] - 0s 946us/step - loss: 94.3955
Epoch 9/50
75/75 [==============================] - 0s 938us/step - loss: 92.1140
Epoch 10/50
75/75 [==============================] - 0s 957us/step - loss: 90.1024
Epoch 11/50
75/75 [==============================] - 0s 955us/step - loss: 88.3592
Epoch 12/50
75/75 [==================

75/75 [==============================] - 0s 1ms/step - loss: 67.2953
Epoch 41/50
75/75 [==============================] - 0s 1ms/step - loss: 67.0740
Epoch 42/50
75/75 [==============================] - 0s 1ms/step - loss: 67.0477
Epoch 43/50
75/75 [==============================] - 0s 1ms/step - loss: 67.0564
Epoch 44/50
75/75 [==============================] - 0s 1ms/step - loss: 66.9833
Epoch 45/50
75/75 [==============================] - 0s 1ms/step - loss: 66.9758
Epoch 46/50
75/75 [==============================] - 0s 1ms/step - loss: 66.9417
Epoch 47/50
75/75 [==============================] - 0s 1ms/step - loss: 66.9412
Epoch 48/50
75/75 [==============================] - 0s 1ms/step - loss: 66.9297
Epoch 49/50
75/75 [==============================] - 0s 1ms/step - loss: 66.8622
Epoch 50/50
75/75 [==============================] - 0s 1ms/step - loss: 66.7842
100	4	8	1.000	2	3	Epoch 1/50
75/75 [==============================] - 0s 1ms/step - loss: 97.9749
Epoch 2/50
75/75 [=====

...

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
75/75 [==============================] - 0s 4ms/step - loss: 94.0699
Epoch 2/50
75/75 [==============================] - 0s 4ms/step - loss: 85.9996
Epoch 3/50
75/75 [==============================] - 0s 4ms/step - loss: 81.4451
Epoch 4/50
75/75 [==============================] - 0s 4ms/step - loss: 78.2221
Epoch 5/50
75/75 [==============================] - 0s 4ms/step - loss: 75.6695
Epoch 6/50
75/75 [==============================] - 0s 4ms/step - loss: 73.7593
Epoch 7/50
75/75 [==============================] - 0s 5ms/step - loss: 72.1906
Epoch 8/50
75/75 [==============================] - 0s 5ms/step - loss: 71.1294
Epoch 9/50
75/75 [==============================] - 0s 5ms/step - loss: 70.2939
Epoch 10/50
75/75 [===========================

75/75 [==============================] - 0s 1ms/step - loss: 65.8778
Epoch 46/50
75/75 [==============================] - 0s 1ms/step - loss: 65.7827
Epoch 47/50
75/75 [==============================] - 0s 1ms/step - loss: 65.6017
Epoch 48/50
75/75 [==============================] - 0s 1ms/step - loss: 65.5144
Epoch 49/50
75/75 [==============================] - 0s 1ms/step - loss: 65.3895
Epoch 50/50
75/75 [==============================] - 0s 1ms/step - loss: 65.2988
Epoch 1/50
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
75/75 [==============================] - 0s 4ms/step - loss: 92.5048
Epoch 2/50
75/75 [==============================] - 0s

75/75 [==============================] - 0s 1ms/step - loss: 66.1648
Epoch 38/50
75/75 [==============================] - 0s 1ms/step - loss: 66.0193
Epoch 39/50
75/75 [==============================] - 0s 1ms/step - loss: 65.8066
Epoch 40/50
75/75 [==============================] - 0s 1ms/step - loss: 65.6836
Epoch 41/50
75/75 [==============================] - 0s 1ms/step - loss: 65.5561
Epoch 42/50
75/75 [==============================] - 0s 1ms/step - loss: 65.4137
Epoch 43/50
75/75 [==============================] - 0s 1ms/step - loss: 65.2937
Epoch 44/50
75/75 [==============================] - 0s 1ms/step - loss: 65.1688
Epoch 45/50
75/75 [==============================] - 0s 1ms/step - loss: 65.0357
Epoch 46/50
75/75 [==============================] - 0s 1ms/step - loss: 64.9318
Epoch 47/50
75/75 [==============================] - 0s 1ms/step - loss: 64.7802
Epoch 48/50
75/75 [==============================] - 0s 1ms/step - loss: 64.6162
Epoch 49/50
75/75 [=====================

75/75 [==============================] - 0s 1ms/step - loss: 67.9342
Epoch 30/50
75/75 [==============================] - 0s 1ms/step - loss: 67.8328
Epoch 31/50
75/75 [==============================] - 0s 1ms/step - loss: 67.6088
Epoch 32/50
75/75 [==============================] - 0s 1ms/step - loss: 67.4790
Epoch 33/50
75/75 [==============================] - 0s 1ms/step - loss: 67.3256
Epoch 34/50
75/75 [==============================] - 0s 1ms/step - loss: 67.2310
Epoch 35/50
75/75 [==============================] - 0s 1ms/step - loss: 67.0339
Epoch 36/50
75/75 [==============================] - 0s 1ms/step - loss: 66.8745
Epoch 37/50
75/75 [==============================] - 0s 1ms/step - loss: 66.6936
Epoch 38/50
75/75 [==============================] - 0s 1ms/step - loss: 66.5555
Epoch 39/50
75/75 [==============================] - 0s 1ms/step - loss: 66.4542
Epoch 40/50
75/75 [==============================] - 0s 1ms/step - loss: 66.3352
Epoch 41/50
75/75 [=====================

AssertionError: 

In [ ]:
#### New Convex Case with Pickle Files  


os.makedirs("results", exist_ok=True)
with open("results/c_notears_paramsweepRealConv.csv", 'w') as out_file:
    print("n\tn_edges\tn_c\tcontext_snr\tk_true\tk", end="\t", file=out_file)
    print("\t".join(["base_{:.3f}".format(x) for x in threshs]), end="\t", file=out_file)
    print("\t".join(["notears_{:.3f}".format(x) for x in threshs]), end='\t', file=out_file)
    print("\t".join(["lioness_{:.3f}".format(x) for x in threshs]), end='\t', file=out_file)
    print("\t".join(["c_notears_{:.3f}".format(x) for x in threshs]), end='\t', file=out_file)
    print("\t".join(["lr_c_notears_{:.3f}".format(x) for x in threshs]), file=out_file)
    
    for arch in range(2, 16):  #number of archetypes
        '''
        data_params = {
    "n": 100,    # number of DAGs
    "n_i": 1,     # number of samples per DAG
    "d" : 16,       # number of vertices in each DAG
    "n_edges": 4,     # expected number of edges in each DAG
    "n_c": 8,    # number of contextual features
    "use_archetypes": True,
    "ensure_convex" : False, # should the archetype be generated such that they form a convex set of DAGs?
    "k_true" : arch,       # number of true archetypes
    "graph_type": "ER",
    'sem_type' : 'gauss',
    'context_snr' : 1.0, # signal-to-noise ratio of the contextual data
    "n_mix": 2
}
        
    ##### This is where the blocked comments began. 
    ##### THIS IS WHERE I BEGIN MASSIVE INDENT  ##################
    
        W, C, X, W_dict, C_dict = gen_data(data_params)
        '''
        W_str = 'Wconv' + str(arch) + '.p'
        C_str = 'Cconv' + str(arch) + '.p'
        X_str = 'Xconv' + str(arch) + '.p'
        W_dictStr = 'W_dictconv' + str(arch) + '.p'
        C_dictStr = 'C_dictconv' + str(arch) + '.p' 
        W = pickle.load(open(W_str, 'rb'))
        C = pickle.load(open(C_str, 'rb'))
        X = pickle.load(open(X_str, 'rb'))
        W_dict = pickle.load(open(W_dictStr, 'rb'))
        C_dict = pickle.load(open(C_dictStr, 'rb'))

        ####### THIS IS ALL OF THE LIONESS STUFF.  #######
        ###### Make sure that the squeeze and split is consistent across models  ########

        ####### I am moving all of this squeezing and unsqueezing and Dataframing South Of the Split  #######



        C_train, C_test, X_train, X_test, W_train, W_test = train_test_split(C, X, W, test_size=0.25)

        ##### # Start Copy From Above   ######
        X = np.squeeze(X)
        X = pd.DataFrame(X)

        #print(X)

        C = pd.DataFrame(C)
        ####### # End Copy From Above
        print("this should all be the same as before")
        #assert False


        C_test =pd.DataFrame(C_test)   # I think this is where the hitch was 
        index = C_test.index
        a_list = list(index)
        print(a_list)



        ##### this is all from test_lioness   ########
        ppi            ='puma/ToyData/ToyPPIData.txt'
        motif          ='puma/ToyData/ToyMotifData.txt'
        expression_data='puma/ToyData/ToyExpressionData.txt'
        lioness_file   ='Travis'
        rm_missing     =False
        output_file    ='panda.npy'
        gt_file        ='panda/test_panda.txt'
        #panda_obj      =Panda(expression_data, motif, ppi, save_tmp=True, remove_missing=rm_missing,
        #                  keep_expression_matrix=bool(lioness_file), modeProcess='legacy', save_memory=False)
        # Set parameters
        #lioness_obj = Lioness(panda_obj, start=1, end=1)
        #lioness_obj = Lioness(panda_obj, data_params, C_train, X_train, start=1, end=1)
        #lioness_obj.save_lioness_results(lioness_file)
        # Read first lioness network
        #res  = np.load('lioness_output/lioness.1.npy')
        #gt = np.load('lioness/lioness.1.npy')
        # Compare to ground truth
        #assert(np.allclose(gt,res))

        #2. Testing Lioness with motif set to None to compute Lioness on coexpression networks
        motif          = None
        # Make sure to keep epxression matrix for next step
        panda_obj      = Panda(expression_data, motif, ppi, save_tmp=True, remove_missing=rm_missing,
                          keep_expression_matrix=True, modeProcess='legacy')
        lioness_obj    = Lioness(panda_obj, data_params, X, a_list, start=1, end=1)
)
        lioness_obj.save_lioness_results(lioness_file)
        # Read first lioness network
        res  = np.load('lioness_output/lioness.1.npy')
        gt   = np.load('lioness/lionessCoexpression.1.npy')
        W_OGLioness = pickle.load(open('lionessParams.pkl', 'rb'))

        W_OGLioness = np.array(W_OGLioness)



        ##### This is where the block comments ended



        #W, C, X, W_dict, C_dict = gen_data(data_params)
        #C_train, C_test, X_train, X_test, W_train, W_test = train_test_split(C, X, W, test_size=0.25)

        notears = NOTEARS({'l1': 1e-3, 'alpha': 1e2, 'rho':1e1, 'gamma':1e1},
                         (data_params["n_c"], 1),
                         (data_params["d"], data_params["d"]))
        print("Fitting Population Model...")
        notears.fit(C_train, X_train, epochs=10, batch_size=1)
        print("Finished Fitting Population Model.")
        pop_model = notears.model.predict(np.expand_dims(C_train[0], 0)).squeeze()


        def not_i(ar, i):
            if i == 0:
                return ar[1:]
            if i == len(ar) - 1:
                return ar[:-1]
            return np.vstack((ar[:i], ar[i+1:]))
        def lioness():
            results = []
            for i in range(data_params["n"]):
                print("{} / {}".format(i, data_params['n']), end='\r')
                notears.model.set_weights([pop_model]) # initialize at pop model
                notears.fit(not_i(C_train, i), not_i(X_train, i), epochs=3, batch_size=1, verbose=0)
                results.append((data_params["n"]-1)*(
                    notears.model.predict(np.expand_dims(C_train[0], 0)).squeeze() - pop_model))
            return results
        #print("Fitting LIONESS...")
        #W_lioness = lioness()
        #print("Finished fitting LIONESS.")
        f1s_lioness   = get_f1s(W_test, W_OGLioness, threshs)
        f1s_baseline = get_f1s(W_test, np.ones_like(W_test), threshs)
        C_test = C_test.to_numpy() 
        f1s_notears  = get_f1s(W_test, [notears.model.predict(np.expand_dims(c, 0)).squeeze() for c in C_test], threshs)



        rank = 5
        for k in range(1, 20):
            print("k={}".format(k), end='\r')
            model_params = {
                "k": k,     # Learned archetype dictionary size
                "encoder_input_shape": (data_params["n_c"], 1),
                "encoder_output_shape": (k,),
                "dict_shape": (k, data_params["d"], data_params["d"]),
                "dict_output_shape": (data_params["d"], data_params["d"]),
                "sample_specific_loss_params": {'l1': 1e-3, 'alpha': 1e2, 'rho': 1e1, 'gamma': 1e1},
                "archetype_loss_params": {'l1': 0, 'alpha': 0, 'rho': 0},
                "learning_rate": 1e-3
            }
            print("{}\t{}\t{}\t{:.3f}\t{}\t{}".format(data_params["n"], data_params["n_edges"], data_params["n_c"],
                                         data_params["context_snr"], data_params["k_true"], 
                                          model_params["k"]), end='\t')

            lr_model_params = {
                "k": k,     # Learned archetype dictionary size
                "encoder_input_shape": (data_params["n_c"], 1),
                "encoder_output_shape": (k,),
                "dict_shape": (k, data_params["d"], rank),
                "dict_output_shape": (data_params["d"], data_params["d"]),
                "sample_specific_loss_params": {'l1': 1e-3, 'alpha': 1e2, 'rho': 1e1, 'gamma': 1e1},
                "archetype_loss_params": {'l1': 0, 'alpha': 0, 'rho': 0},
                "learning_rate": 1e-3
            }
            lr_c_notears = LRContextualNOTEARS(lr_model_params["encoder_input_shape"],
                                          lr_model_params["encoder_output_shape"],
                                          lr_model_params["dict_shape"],
                                         lr_model_params["sample_specific_loss_params"],
                                         lr_model_params["archetype_loss_params"],
                                         lr_model_params["learning_rate"])
            lr_c_notears.fit(C_train, X_train, batch_size=1, epochs=50)

            c_notears = ContextualNOTEARS(model_params["encoder_input_shape"],
                                          model_params["encoder_output_shape"],
                                          model_params["dict_shape"],
                                         model_params["sample_specific_loss_params"],
                                         model_params["archetype_loss_params"],
                                         model_params["learning_rate"])
            c_notears.fit(C_train, X_train, batch_size=1, epochs=50)

            # Measure recovery of sample-specific networks
            W_test_pred = [c_notears.model.predict(np.expand_dims(c, 0)).squeeze() for c in C_test]
            f1s = get_f1s(W_test, W_test_pred, threshs)
            W_test_pred_lr = [lr_c_notears.model.predict(np.expand_dims(c, 0)).squeeze() for c in C_test]
            f1s_lr = get_f1s(W_test, W_test_pred_lr, threshs)
            print("{}\t{}\t{}\t{:.3f}\t{}\t{}".format(data_params["n"], data_params["n_edges"], data_params["n_c"],
                                         data_params["context_snr"], data_params["k_true"], 
                                          model_params["k"]), end='\t', file=out_file)
            print("\t".join(["{:.3f}".format(x) for x in f1s_baseline]), end='\t', file=out_file)
            print("\t".join(["{:.3f}".format(x) for x in f1s_notears]), end='\t', file=out_file)
            print("\t".join(["{:.3f}".format(x) for x in f1s_lioness]), end='\t', file=out_file)
            print("\t".join(["{:.3f}".format(x) for x in f1s]), end='\t', file=out_file)
            print("\t".join(["{:.3f}".format(x) for x in f1s_lr]), file=out_file)
        
        










In [ ]:
#######  CONVEX CASE   ##########

os.makedirs("results", exist_ok=True)
with open("results/c_notears_paramsweepNonConv.csv", 'w') as out_file:
    print("n\tn_edges\tn_c\tcontext_snr\tk_true\tk", end="\t", file=out_file)
    print("\t".join(["base_{:.3f}".format(x) for x in threshs]), end="\t", file=out_file)
    print("\t".join(["notears_{:.3f}".format(x) for x in threshs]), end='\t', file=out_file)
    print("\t".join(["lioness_{:.3f}".format(x) for x in threshs]), end='\t', file=out_file)
    print("\t".join(["c_notears_{:.3f}".format(x) for x in threshs]), end='\t', file=out_file)
    print("\t".join(["lr_c_notears_{:.3f}".format(x) for x in threshs]), file=out_file)
    
    for arch in range(15, 30):  #number of archetypes
        data_params = {
    "n": 100,    # number of DAGs
    "n_i": 1,     # number of samples per DAG
    "d" : 16,       # number of vertices in each DAG
    "n_edges": 4,     # expected number of edges in each DAG
    "n_c": 8,    # number of contextual features
    "use_archetypes": True,
    "ensure_convex" : True, # should the archetype be generated such that they form a convex set of DAGs?
    "k_true" : arch,       # number of true archetypes
    "graph_type": "ER",
    'sem_type' : 'gauss',
    'context_snr' : 1.0, # signal-to-noise ratio of the contextual data
    "n_mix": 2
}
        
    ##### This is where the blocked comments began.      
        W, C, X, W_dict, C_dict = gen_data(data_params)
        W_str = 'Wconv' + str(arch) + '.p'
        C_str = 'Cconv' + str(arch) + '.p'
        X_str = 'Xconv' + str(arch) + '.p'
        W_dictStr = 'W_dictconv' + str(arch) + '.p'
        C_dictStr = 'C_dictconv' + str(arch) + '.p'


        pickle.dump(W, open(W_str, 'wb'))
        pickle.dump(C, open(C_str, 'wb'))
        pickle.dump(X, open(X_str, 'wb'))
        pickle.dump(W_dict, open(W_dictStr, 'wb'))
        pickle.dump(C_dict, open(C_dictStr, 'wb'))
        print("got through one")
    
    
    
    print("got through gen data")
    
    ####### THIS IS ALL OF THE LIONESS STUFF.  #######
    ###### Make sure that the squeeze and split is consistent across models  ########
    
    ####### I am moving all of this squeezing and unsqueezing and Dataframing South Of the Split  #######
    #print(X.shape)
    #X = np.squeeze(X)
    #X = pd.DataFrame(X)
    
    #print(X)
    
    #C = pd.DataFrame(C)
    #print("did we get into this thing?")
    #assert False
    #W = pd.DataFrame(W)   #1000 x 128 x 128
    
    
    C_train, C_test, X_train, X_test, W_train, W_test = train_test_split(C, X, W, test_size=0.25)
    
    ##### # Start Copy From Above   ######
    X = np.squeeze(X)
    X = pd.DataFrame(X)
    
    #print(X)
    
    C = pd.DataFrame(C)
    ####### # End Copy From Above
    print("this should all be the same as before")
    #assert False
    
    
    C_test =pd.DataFrame(C_test)   # I think this is where the hitch was 
    index = C_test.index
    a_list = list(index)
    print(a_list)
    
    #print(C_train)
    #print(C_train.shape)   #750 x 8    between 0.0 and 1.0
    #print("it got here mofos")
    #print(W_test)   #centered around 0
    #print(W_test.shape)   # (250, 128, 128)
    #print(X_train)
    #print(X_train.shape)    #(750, 1, 128)
    #assert False
    #X = np.squeeze(X)
    #print(X.shape)
    #assert False
    #W_lioness = lioness()
    #print(W_test)
    #print(type(W_test))
    #assert False
    
    ##### this is all from test_lioness   ########
    ppi            ='puma/ToyData/ToyPPIData.txt'
    motif          ='puma/ToyData/ToyMotifData.txt'
    expression_data='puma/ToyData/ToyExpressionData.txt'
    lioness_file   ='Travis'
    rm_missing     =False
    output_file    ='panda.npy'
    gt_file        ='panda/test_panda.txt'
    #panda_obj      =Panda(expression_data, motif, ppi, save_tmp=True, remove_missing=rm_missing,
    #                  keep_expression_matrix=bool(lioness_file), modeProcess='legacy', save_memory=False)
    # Set parameters
    #lioness_obj = Lioness(panda_obj, start=1, end=1)
    #lioness_obj = Lioness(panda_obj, data_params, C_train, X_train, start=1, end=1)
    #lioness_obj.save_lioness_results(lioness_file)
    # Read first lioness network
    #res  = np.load('lioness_output/lioness.1.npy')
    #gt = np.load('lioness/lioness.1.npy')
    # Compare to ground truth
    #assert(np.allclose(gt,res))

    #2. Testing Lioness with motif set to None to compute Lioness on coexpression networks
    motif          = None
    # Make sure to keep epxression matrix for next step
    panda_obj      = Panda(expression_data, motif, ppi, save_tmp=True, remove_missing=rm_missing,
                      keep_expression_matrix=True, modeProcess='legacy')
    lioness_obj    = Lioness(panda_obj, data_params, X, a_list, start=1, end=1)
    #lioness_obj    = Lioness(panda_obj, data_params, C_train, X_train, start=1, end=1)
    lioness_obj.save_lioness_results(lioness_file)
    # Read first lioness network
    res  = np.load('lioness_output/lioness.1.npy')
    gt   = np.load('lioness/lionessCoexpression.1.npy')
    W_OGLioness = pickle.load(open('jigglydiggly.pkl', 'rb'))
    
    W_OGLioness = np.array(W_OGLioness)
    print("let's see if we got this far")
    #assert False

    
    
    ##### This is where the block comments ended
        
        
    
    #W, C, X, W_dict, C_dict = gen_data(data_params)
    #C_train, C_test, X_train, X_test, W_train, W_test = train_test_split(C, X, W, test_size=0.25)
    
    notears = NOTEARS({'l1': 1e-3, 'alpha': 1e2, 'rho':1e1, 'gamma':1e1},
                     (data_params["n_c"], 1),
                     (data_params["d"], data_params["d"]))
    print("Fitting Population Model...")
    notears.fit(C_train, X_train, epochs=10, batch_size=1)
    print("Finished Fitting Population Model.")
    pop_model = notears.model.predict(np.expand_dims(C_train[0], 0)).squeeze()
    
    
    def not_i(ar, i):
        if i == 0:
            return ar[1:]
        if i == len(ar) - 1:
            return ar[:-1]
        return np.vstack((ar[:i], ar[i+1:]))
    def lioness():
        results = []
        for i in range(data_params["n"]):
            print("{} / {}".format(i, data_params['n']), end='\r')
            notears.model.set_weights([pop_model]) # initialize at pop model
            notears.fit(not_i(C_train, i), not_i(X_train, i), epochs=3, batch_size=1, verbose=0)
            results.append((data_params["n"]-1)*(
                notears.model.predict(np.expand_dims(C_train[0], 0)).squeeze() - pop_model))
        return results
    #print("Fitting LIONESS...")
    #W_lioness = lioness()
    #print("Finished fitting LIONESS.")
    f1s_lioness   = get_f1s(W_test, W_OGLioness, threshs)
    f1s_baseline = get_f1s(W_test, np.ones_like(W_test), threshs)
    C_test = C_test.to_numpy() 
    f1s_notears  = get_f1s(W_test, [notears.model.predict(np.expand_dims(c, 0)).squeeze() for c in C_test], threshs)
    
    print("DDDDDIIIIIDDDD WE GET THROUGH THE F1 SCORES?")

    rank = 5
    for k in range(1, 20):
        print("k={}".format(k), end='\r')
        model_params = {
            "k": k,     # Learned archetype dictionary size
            "encoder_input_shape": (data_params["n_c"], 1),
            "encoder_output_shape": (k,),
            "dict_shape": (k, data_params["d"], data_params["d"]),
            "dict_output_shape": (data_params["d"], data_params["d"]),
            "sample_specific_loss_params": {'l1': 1e-3, 'alpha': 1e2, 'rho': 1e1, 'gamma': 1e1},
            "archetype_loss_params": {'l1': 0, 'alpha': 0, 'rho': 0},
            "learning_rate": 1e-3
        }
        print("{}\t{}\t{}\t{:.3f}\t{}\t{}".format(data_params["n"], data_params["n_edges"], data_params["n_c"],
                                     data_params["context_snr"], data_params["k_true"], 
                                      model_params["k"]), end='\t')

        lr_model_params = {
            "k": k,     # Learned archetype dictionary size
            "encoder_input_shape": (data_params["n_c"], 1),
            "encoder_output_shape": (k,),
            "dict_shape": (k, data_params["d"], rank),
            "dict_output_shape": (data_params["d"], data_params["d"]),
            "sample_specific_loss_params": {'l1': 1e-3, 'alpha': 1e2, 'rho': 1e1, 'gamma': 1e1},
            "archetype_loss_params": {'l1': 0, 'alpha': 0, 'rho': 0},
            "learning_rate": 1e-3
        }
        lr_c_notears = LRContextualNOTEARS(lr_model_params["encoder_input_shape"],
                                      lr_model_params["encoder_output_shape"],
                                      lr_model_params["dict_shape"],
                                     lr_model_params["sample_specific_loss_params"],
                                     lr_model_params["archetype_loss_params"],
                                     lr_model_params["learning_rate"])
        lr_c_notears.fit(C_train, X_train, batch_size=1, epochs=50)

        c_notears = ContextualNOTEARS(model_params["encoder_input_shape"],
                                      model_params["encoder_output_shape"],
                                      model_params["dict_shape"],
                                     model_params["sample_specific_loss_params"],
                                     model_params["archetype_loss_params"],
                                     model_params["learning_rate"])
        c_notears.fit(C_train, X_train, batch_size=1, epochs=50)

        # Measure recovery of sample-specific networks
        W_test_pred = [c_notears.model.predict(np.expand_dims(c, 0)).squeeze() for c in C_test]
        f1s = get_f1s(W_test, W_test_pred, threshs)
        W_test_pred_lr = [lr_c_notears.model.predict(np.expand_dims(c, 0)).squeeze() for c in C_test]
        f1s_lr = get_f1s(W_test, W_test_pred_lr, threshs)
        print("{}\t{}\t{}\t{:.3f}\t{}\t{}".format(data_params["n"], data_params["n_edges"], data_params["n_c"],
                                     data_params["context_snr"], data_params["k_true"], 
                                      model_params["k"]), end='\t', file=out_file)
        print("\t".join(["{:.3f}".format(x) for x in f1s_baseline]), end='\t', file=out_file)
        print("\t".join(["{:.3f}".format(x) for x in f1s_notears]), end='\t', file=out_file)
        print("\t".join(["{:.3f}".format(x) for x in f1s_lioness]), end='\t', file=out_file)
        print("\t".join(["{:.3f}".format(x) for x in f1s]), end='\t', file=out_file)
        print("\t".join(["{:.3f}".format(x) for x in f1s_lr]), file=out_file)
        
        
print(" ALLLL THROUGH CONVEX WOOHOO  ")
assert False


In [ ]:
import pandas as pd
df = pd.read_csv("results/c_notears_paramsweep.csv", header=0, sep='\t')
df.head()

In [ ]:
for i, thresh in enumerate(threshs):
    fig = plt.figure(figsize=(8,8))
    plt.scatter(df['k'], df['base_{:.3f}'.format(thresh)], label='All 1s', color='black')
    #plt.scatter(df['k'], df['notears_{:.3f}'.format(thresh)], label='NOTEARS', color='blue')
    #plt.axhline(f1s_base[i], label='All 1s')
    plt.axhline(f1s_notears[i], linestyle='--', color='orange', label='NOTEARS')
    plt.axhline(f1s_lioness[i], linestyle='--', color='purple', label='LIONESS')
    #plt.scatter(df['k'], df['lioness_{:.3f}'.format(lioness)], label='LIONESS', color='orange')
    plt.scatter(df['k'], df['c_notears_{:.3f}'.format(thresh)], label='C-NOTEARS', color='red')
    plt.scatter(df['k'], df['lr_c_notears_{:.3f}'.format(thresh)], label='LR-C-NOTEARS', color='blue')
    plt.legend(fontsize=18)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.xlabel("K", fontsize=28)
    plt.ylabel("Mean F1-Score of SS Networks", fontsize=24)
    plt.title("Thresholded at {}".format(thresh), fontsize=24)

In [ ]:
# IF archetypes exist,
# Measure recovery of archetypes
# TODO: Have to match up archetypes to dictionary, no guarantee of order.
threshs = [0.0, 0.001, 0.01, 0.02, 0.03, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
W_dict_pred = c_notears.explainer.trainable_variables[0].numpy()
f1s_baseline = get_f1s(W_dict, np.ones_like(W_dict_pred), thresh)

plt.plot(threshs, f1s)
plt.xlabel("Threshold")
plt.ylabel("F1-Score of Archetype Edge Recovery")

In [ ]:
# Visualize a prediction
#y_pred = trim_params(y_pred)
#plt.imshow(utils.trim_params(w_test_pred[0]), cmap='coolwarm', vmin=-2, vmax=2)
plt.imshow(w_test_pred[0], cmap='coolwarm', vmin=-2, vmax=2)
plt.title(r"$W_{i=185}$ Predicted")
plt.colorbar()
plt.show()
plt.imshow(w_test[0], cmap='coolwarm', vmin=-2, vmax=2)
plt.title(r"$W_{i=185}$ Truth")
plt.colorbar()
plt.show()

In [ ]:
# Print data shapes and visualize true archetypes
print(f"subtypes {subtypes.shape}, W_n {W_n.shape}, c_n {c_n.shape}, X_n {X_n.shape}")
for i in range(len(W_k)):
    print(f"true archetype {i}")
    plt.imshow(W_k[i], cmap='coolwarm', vmin=-2, vmax=2)
    plt.colorbar()
    plt.show()

In [ ]:
# Visualize the learned archetypes
W_k_pred = c_notears.explainer.trainable_variables[0].numpy()
for i in range(k):
    print(f"pred {i}")
    plt.imshow(W_k_pred[i], cmap='coolwarm', vmin=-2, vmax=2)
    plt.colorbar()
    plt.show()
for i in range(k):
    print(f"true {i}")
    plt.imshow(W_k[i], cmap='coolwarm', vmin=-2, vmax=2)
    plt.colorbar()
    plt.show()

In [ ]:
# Display the predicted subtype and real subtype
encoder(e_n[999:2000]).numpy(), subtypes[999:2000]

In [ ]:
print([f1_mat(W_k[i], W_k_pred[i], 0.0, 0.2) for i in range(k)])
print(np.mean([f1_mat(W_k[i], W_k_pred[i], 0.0, 0.2) for i in range(k)]))

In [ ]:
plt.imshow(W_k[0], cmap='coolwarm', vmin=-2, vmax=2)

In [ ]:
# Check the MSE of some test sample for validation
y_pred = c_notears.model.predict(e_n[999:1000])
y_true = W_n[999:1000][0]
print(f"mse| pred: {np.mean(np.sum(np.square(y_pred-y_true)))}, baseline: {np.mean(np.sum(np.square(np.zeros(y_pred.shape)-y_true)))}")